In [2]:
import pandas as pd
import numpy as np

# 📂 Directorio de la carpeta CLEAN
clean_path = "/Users/chants/Desktop/DM_TEL_cucalon/Data/clean/"

# 📌 --- CARGAR DATOS LIMPIOS ---
df_users = pd.read_csv(clean_path + "megaline_users.csv")
df_calls = pd.read_csv(clean_path + "megaline_calls.csv")
df_messages = pd.read_csv(clean_path + "megaline_messages.csv")
df_internet = pd.read_csv(clean_path + "megaline_internet.csv")
df_plans = pd.read_csv(clean_path + "megaline_plans.csv")

print("✅ Datos cargados correctamente.")

# 📌 --- CREAR DIMENSIONES ---

## 🌟 Dimensión de Usuarios (dim_users)
dim_users = df_users[["user_id", "first_name", "last_name", "age", "city", "reg_date", "plan", "churn_status"]]

## 🌟 Dimensión de Planes (dim_plans)
dim_plans = df_plans.copy()

## 🌟 Dimensión de Tiempo (dim_time)
df_calls["call_date"] = pd.to_datetime(df_calls["call_date"])
df_messages["message_date"] = pd.to_datetime(df_messages["message_date"])
df_internet["session_date"] = pd.to_datetime(df_internet["session_date"])

# Generamos una dimensión de tiempo que abarca el año 2018
dim_time = pd.DataFrame({
    "date": pd.date_range(start="2018-01-01", end="2018-12-31", freq="D")
})
dim_time["month"] = dim_time["date"].dt.to_period("M")
dim_time["year"] = dim_time["date"].dt.year

print("✅ Dimensiones creadas.")

#  --- PROCESAR TABLA DE HECHOS (fact_billing) ---
df_calls["month"] = df_calls["call_date"].dt.to_period("M")
df_messages["month"] = df_messages["message_date"].dt.to_period("M")
df_internet["month"] = df_internet["session_date"].dt.to_period("M")

## Crear base de usuarios y meses para no perder datos
all_users = df_users["user_id"].unique()
all_months = pd.period_range("2018-01", "2018-12", freq="M")
base_df = pd.MultiIndex.from_product([all_users, all_months], names=["user_id", "month"]).to_frame(index=False)

## --- Agregar uso mensual de llamadas ---
calls_usage = df_calls.groupby(["user_id", "month"])["duration"].sum().reset_index()
calls_usage.rename(columns={"duration": "total_calls_duration"}, inplace=True)

## --- Agregar uso mensual de mensajes ---
messages_usage = df_messages.groupby(["user_id", "month"]).size().reset_index(name="total_messages")

## --- Agregar uso mensual de internet ---
internet_usage = df_internet.groupby(["user_id", "month"])["mb_used"].sum().reset_index()

## --- Unir todos los datos con un OUTER JOIN (para no perder usuarios) ---
fact_billing = base_df.merge(calls_usage, on=["user_id", "month"], how="left")
fact_billing = fact_billing.merge(messages_usage, on=["user_id", "month"], how="left")
fact_billing = fact_billing.merge(internet_usage, on=["user_id", "month"], how="left")

## 🛠️ Reemplazar NaN por 0 en valores de uso
fact_billing.fillna(0, inplace=True)

## --- CALCULAR COSTOS ---

# Unir con información de planes
fact_billing = fact_billing.merge(df_users[["user_id", "plan"]], on="user_id", how="left")
fact_billing = fact_billing.merge(df_plans, left_on="plan", right_on="plan_name", how="left")

# Calcular el costo total por mes (no necesitamos calcular costos individuales si solo necesitamos el total)
fact_billing["total_cost"] = (
    fact_billing["usd_monthly_pay"] + 
    np.where(
        fact_billing["total_calls_duration"] > fact_billing["minutes_included"],
        (fact_billing["total_calls_duration"] - fact_billing["minutes_included"]) * fact_billing["usd_per_minute"],
        0
    ) + 
    np.where(
        fact_billing["total_messages"] > fact_billing["messages_included"],
        (fact_billing["total_messages"] - fact_billing["messages_included"]) * fact_billing["usd_per_message"],
        0
    ) + 
    np.where(
        fact_billing["mb_used"] > fact_billing["mb_per_month_included"],
        ((fact_billing["mb_used"] - fact_billing["mb_per_month_included"]) / 1024) * fact_billing["usd_per_gb"],
        0
    )
)

# Seleccionar columnas finales (sin redundancias)
fact_billing = fact_billing[[
    "user_id", "month", "total_calls_duration", "total_messages", "mb_used", "total_cost"
]]

print("✅ Tabla de hechos creada correctamente con OUTER JOIN.")

# 📌 --- EXPORTAR A CSV ---
fact_billing.to_csv(clean_path + "fact_billing.csv", index=False)
dim_users.to_csv(clean_path + "dim_users.csv", index=False)
dim_plans.to_csv(clean_path + "dim_plans.csv", index=False)
dim_time.to_csv(clean_path + "dim_time.csv", index=False)

print("✅ Tablas exportadas correctamente en:", clean_path)



✅ Datos cargados correctamente.
✅ Dimensiones creadas.
✅ Tabla de hechos creada correctamente con OUTER JOIN.
✅ Tablas exportadas correctamente en: /Users/chants/Desktop/DM_TEL_cucalon/Data/clean/


Star Schema. Tabla de Hechos (fact_consumo) contendrá los datos de consumo de cada usuario en cada período.
Tablas de Dimensiones (dim_usuarios, dim_planes, dim_tiempo) permitirán realizar consultas eficientes.